In [ ]:
import numpy as np
import re

with open('sample.txt', 'r') as f:
  lines = f.read().splitlines()

In [ ]:
def parse_line(line):
  (src, dests) = re.search(r'(.+):\s?(.*)', line).groups()
  return (src, re.split(r'\s+', dests))

rels = list(map(parse_line, lines))

def get_edge_names(rels):
  edges = set()
  for (src, dests) in rels:
    edges.add(src)
    for dest in dests:
      edges.add(dest)
  return sorted(edges)


# Build affinity matrix
def get_affinity_matrix(rels):
  edge_names = get_edge_names(rels)
  A = np.zeros((len(edge_names), len(edge_names)), dtype=int)

  for (src, dests) in rels:
    i_src = edge_names.index(src)
    for dest in dests:
      i_dest = edge_names.index(dest)
      A[i_src, i_dest] = 1
      A[i_dest, i_src] = 1

  return A

In [ ]:
# We'll run Karger's algorithm to estimate a global min-cut.

while True:
  G = get_affinity_matrix(rels)
  G = np.triu(G)

  edges = list(np.transpose(np.nonzero(G)))

  # We'll represent "merged" vertices by assigning them the same ID. Initially,
  # no vertices have been merged, so each vertex has its own ID.
  vertex_group_ids = np.arange(G.shape[0])

  while len(np.unique(vertex_group_ids)) > 2:
    #print(f'Running for {len(np.unique(vertex_group_ids))} vertices...')

    idx = np.random.randint(len(edges))
    i, j = edges.pop(idx)
    #print(f'  i, j: {i}, {j}')

    i_vertices = (vertex_group_ids == vertex_group_ids[i])
    j_vertices = (vertex_group_ids == vertex_group_ids[j])

    # Remove all edges between vertices in these groups
    G[np.ix_(i_vertices, j_vertices)] = 0
    G[np.ix_(j_vertices, i_vertices)] = 0

    # Reassign all vertices in group "j" to the other group
    vertex_group_ids[j_vertices] = vertex_group_ids[i]


  min_cut_size = np.sum(np.triu(G))
  group_counts = np.unique(vertex_group_ids, return_counts=True)[1]

  print(f'min_cut_size: {min_cut_size}   group_counts: {group_counts}')

  if min_cut_size == 3:
    break